# Building a Ratings Matrix

The purpose of this workbook is to build a user-item ratings matrix which will be used for a collaborative based recommender system.

The final output will be a ratings matrix which will be saved as a .pkl (pickle) file to be used in future workbooks.

***

In [2]:
# Import Python libraries as needed
import numpy as np
import pandas as pd

In [3]:
# Read in saved data-frames from previous workbook
business = pd.read_pickle('data/user/business.pkl')
review = pd.read_pickle('data/user/review.pkl')

In [4]:
# Quick overview of business table
print(business.shape)
business.head(3)

(6558, 12)


,business_id,name,address,city,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
0,0QjROMVW9ACKjhSEfHqNCQ,Mi Mi Restaurant,688 Gerrard Street E,Toronto,M4M 1Y3,43.666376,-79.348773,4.0,116,"{'RestaurantsTakeOut': 'True', 'Alcohol': 'u'b...","Vietnamese, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
1,8k62wYhDVq1-652YbJi5eg,Tim Hortons,90 Adelaide Street W,Toronto,M5H 3V9,43.649859,-79.382060,3.0,8,"{'OutdoorSeating': 'False', 'RestaurantsDelive...","Bagels, Donuts, Food, Cafes, Coffee & Tea, Res...",None
2,34FYKG4pHNXbM9ZRRiJaGw,Tim Hortons,939 Eglinton Avenue E,East York,M4G 4E8,43.713463,-79.360166,1.5,3,"{'RestaurantsPriceRange2': '1', 'WiFi': ''free...","Cafes, Donuts, Food, Coffee & Tea, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [5]:
# Quick overview of review table
print(review.shape)
review.head(3)

(378098, 9)


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Tyx7AxYQfSRnBFUIXUJwFA,HJECayULRM-6xh2GCCvLiA,l-nL4BmhzpZjcavooO48PQ,4,2,1,1,ORDER In (Delivery) Review\n\nI discovered thi...,2015-10-17 01:38:13
1,KBCqABnVOKakrHgJexftCw,n8sZMywJ2q-A12BIewfCFA,l-nL4BmhzpZjcavooO48PQ,5,1,0,0,"The Wokker is the best, most authentic Chinese...",2012-12-04 23:28:29
2,G2OVt3FJRvZOSneyKiJiCg,F3Mu8M7bGbW960OwTvPnSw,l-nL4BmhzpZjcavooO48PQ,1,0,0,0,In Toronto on business. Ordered for room servi...,2018-06-19 22:21:31


In [6]:
# Confirm there are no records in the review table with missing star ratings
review['stars'].isna().value_counts()

False    378098
Name: stars, dtype: int64

***

In [7]:
# Create a 0-n index that can be called as a series for each unique 'user_id' value
unique_user = review['user_id'].unique()

# Transform series into a data-frame
unique_user = pd.DataFrame(unique_user)

# Reset the index such that the index is copied into a different column labelled 'user_num'
unique_user = unique_user.reset_index().rename(columns = {'index':'user_num',0:'user_id'})

# Review final result
unique_user

# Save unique_user df to pickle file for use in future workbook
unique_user.to_pickle('data/user/unique_user.pkl')

In [8]:
# Create a 0-n index that can be called as a series for each unique 'business_id' value
unique_business = review['business_id'].unique()

# Transform series into a data-frame
unique_business = pd.DataFrame(unique_business)

# Reset the index such that the index is copied into a different column labelled 'business_num'
unique_business = unique_business.reset_index().rename(columns = {'index':'business_num',0:'business_id'})

# Review final result
unique_business

# Save unique_user df to pickle file for use in future workbook
unique_business.to_pickle('data/user/unique_business.pkl')

In [11]:
# Capture the length of each unique user and business lists
num_user = len(unique_user)
num_business = len(unique_business)

# Print both values
print('unique users:', num_user, '\nunique businesses:', num_business)

unique users: 96532 
unique businesses: 6558


In [13]:
%%time
# Capture cell execution time (approx. 1 min.)

# Create an empty array with a size that is equal to number of unique users x number of unique businesses
ratings_array = np.full((num_user, num_business), np.nan)

# Review final result
ratings_array

CPU times: user 940 ms, sys: 1.15 s, total: 2.09 s
Wall time: 2.48 s


array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [14]:
# Create seperate data-frame for user-item ratings matrix
mini_review = review.loc[:,['user_id','business_id','stars','date']].copy()

# Sort data frame by most recent to least recent date (in order to assist with dropping duplicates)
mini_review.sort_values(by='date', ascending = False, inplace = True)

# Quick overview of the 'mini_review' data-frame
print(mini_review.shape)
mini_review.head(3)

(378098, 4)


,user_id,business_id,stars,date
107560,rLl5jjjUK1crVfzbXoQrTA,vmQKPYE46edrPQDc1vBMDA,4,2019-12-13 15:34:14
372025,t67b86Sjg_s7PqfZCImR2Q,taSRrjQs0O7pnzd5oiQx7Q,5,2019-12-13 14:30:31
64338,1X5Ws7m_Cwh4DVBestTHeg,bFzFGDQTLsAs1WKA_EGjZA,5,2019-12-13 14:06:32


In [16]:
# Drop all duplicate values such that there is only one star rating for every unique user-business combination
# By default the first value is kept for each set of duplicate values
# Since we previously sorted the review by date/time, we can be assured that only the most recent review for each business has been kept for each user
mini_review.drop_duplicates(['user_id','business_id'], inplace = True)
mini_review.shape

(367390, 4)

In [17]:
# Lookup the unique integer based index values for each unique user_id and business_id
mini_review = pd.merge(mini_review, unique_user, how = 'left')
mini_review = pd.merge(mini_review, unique_business, how = 'left')
mini_review.head(3)

,user_id,business_id,stars,date,user_num,business_num
0,rLl5jjjUK1crVfzbXoQrTA,vmQKPYE46edrPQDc1vBMDA,4,2019-12-13 15:34:14,1669,1678
1,t67b86Sjg_s7PqfZCImR2Q,taSRrjQs0O7pnzd5oiQx7Q,5,2019-12-13 14:30:31,44671,6513
2,1X5Ws7m_Cwh4DVBestTHeg,bFzFGDQTLsAs1WKA_EGjZA,5,2019-12-13 14:06:32,32045,645


In [26]:
# Confirm that user_num and business_num values have been applied successfully and that there are no missing values
print(mini_review['business_num'].isna().value_counts(),'\n')
print(mini_review['user_num'].isna().value_counts())

False    367390
Name: business_num, dtype: int64 

False    367390
Name: user_num, dtype: int64


In [32]:
# Showing an example record by one user designated as 'user_num 28532'
mini_review[mini_review['user_num'] == 28532]

,user_id,business_id,stars,date,user_num,business_num
305804,2Lx9Bp34C_GfzDJFI4U9TA,dd3GZs6zW2t7BM0cbxAzvg,4,2014-01-13 01:29:35,28532,650


In [50]:
# Transfer the star rating from the 'mini_review' df into the 'ratings_array' df

# Loop over the number of rows within the 'mini_review' data-frame
for row in mini_review.itertuples():
    
    # Capture the value under the 'user_num' column
    user_index = row[5]
    
    # Capture the value under the 'business_num' column
    business_index = row[6]
    
    # Capture the rating under the 'stars' column
    rating_value = row[3]
    
    # Update the 'ratings_array' using the 'user_num' and 'business_num' as the index value in order to store the 'stars' value
    ratings_array[user_index, business_index] = rating_value
    
# Transform the 'ratings_array' into a data-frame and then output a few of its records    
ratings_matrix = pd.DataFrame(ratings_array)
ratings_matrix

,0,1,2,3,4,5,6,7,8,9,...,6548,6549,6550,6551,6552,6553,6554,6555,6556,6557
0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
96530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# Select any 'user_num' value to act as the selected user (for use in future functions)
current_user = 5

In [52]:
# Create a boolean series indicating the non-null columns for the given row number (i.e. the 'current_user')
user_pick_non_blanks = ratings_df.loc[current_user,].isna() == False

# Pull all the non-null values from the ratings table for the 'current_user'
ratings_matrix.loc[current_user,][user_pick_non_blanks]

0       4.0
38      5.0
59      3.0
126     4.0
132     3.0
       ... 
6160    5.0
6192    3.0
6228    4.0
6257    2.0
6399    1.0
Name: 5, Length: 118, dtype: float64

In [54]:
# Save the 'ratings_matrix' data-frame to a .pkl file
ratings_matrix.to_pickle('data/user/ratings_matrix.pkl')